In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-1.7_2019.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

## What the structure of the final DataFrame is supposed to look like after stitching(concatinating)?

| tableA   | tableB   | tableC   |
| -------- | -------- | -------- |
| Page1    | Page3    | Page5    |
| Page2    | Page4    | Page6    |

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-1.7_2019.pdf'

In [5]:
#Scraping Pages 1 to 3
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=["21.162649434571893,728.5539356601939,582.9347980613894,115.0160479455716"],
                          columns=["44.2491760904685,174.11088852988692,207.7787399030695,246.25628432956384,274.15250403877224,307.8203554119548,346.29789983844915,381.88962843295644,409.7858481421648,447.30145395799684,482.8931825525041,518.4849111470114,546.3811308562198"],
                          split_text=True,
                          row_tol=12,
                          strip_text='\n\t',
                          pages="1-3"
                         )

In [6]:
tables

<TableList n=3>

In [7]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [8]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
printnAllTables(tables)

Page No:  1
         0                                                  1       2   \
0   Sl. No.                                              Cause  Male F   
1                                                                        
2       (1)                                                (2)     (3)   
3         1                                          Air Crash       0   
4         2                                     Ship Accidents       0   
5         3                      Collapse of Structure (Total)     141   
6            3.1 Collapse of Dwelling House/Residential Bui...     105   
7                3.2 Collapse of Official/ Commercial Building       2   
8                                          3.3 Collapse of Dam       0   
9                                      3.4 Collapse of  Bridge       0   
10                                                  3.5 Others      34   
11        4                                   Drowning (Total)    3257   
12                        

In [9]:
# Removes Garbage Rows that may have been detected Before the 'Air Crash' or 'Sudden Deaths (Total)' row
# AND
# Removes Garbage Rows that may have been detected after the 'TOTAL' or 'Stampede' row
def clean(table):
    while(table.df.iloc[0,1] not in  ['Air Crash','Sudden Deaths (Total)']):
        table.df = table.df.iloc[1:]
    while(table.df.iloc[-1,1] not in  ['Total', 'Stampede']):
        table.df = table.df.iloc[:-1]
    # The Return line is unnecessary due to all changes being reflected in the original references
    return table

In [10]:
for each_table in tables:
    clean(each_table)

In [11]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [12]:
#Scraping Page 4
table4 = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=["21.162649434571893,713.1626351882904,582.9347980613894,385.13554388084685"],
                          columns=["44.2491760904685,167.37731825525043,204.8929240710824,243.37046849757675,267.4189337641357,305.8964781906301,344.3740226171244,382.8515670436188,409.7858481421648,446.33951534733444,479.04542810985464,517.522972536349,544.4572536348951"],
                          split_text=True,
                          row_tol=12,
                          strip_text='\n\t',
                          pages="4"
                         )
clean(table4[0])

<Table shape=(22, 14)>

In [13]:
#Scraping Page 5
table5 = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=["36.55366720516963,736.2495858961457,542.5333764135703,157.1519056407761"],
                          columns=["59.640193861066244,215.47424878836836,279.9241357027464,342.4501453957997,408.82390953150247,492.5125686591277"],
                          split_text=True,
                          row_tol=12,
                          strip_text='\n\t',
                          pages="5"
                         )
clean(table5[0])

<Table shape=(40, 7)>

In [14]:
#Scraping Page 6
table6 = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=["36.55366720516963,697.7392773354907,544.4572536348951,357.20675439462553"],
                          columns=["63.487948303715676,211.62649434571892,279.9241357027464,343.41208400646207,412.6716639741519,492.5125686591277"],
                          split_text=True,
                          row_tol=12,
                          strip_text='\n\t',
                          pages="6"
                         )
clean(table6[0])

<Table shape=(23, 7)>

In [15]:
df1 = tables[0].df
df2 = tables[1].df
df3 = tables[2].df
df4 = table4[0].df
df5 = table5[0].df
df6 = table6[0].df

In [16]:
tableA = pd.concat([df1,df2],axis=0)
tableB = pd.concat([df3,df4],axis=0)
tableC = pd.concat([df5,df6],axis=0)

In [17]:
# To Rename the columns accordingly
tableAColumnList = [
    ("Sl. No.",''),
    ("State/UT",''), 
    ('Below 14 years','Male'),
    ('Below 14 years','Female'),
    ('Below 14 years','Transgender'),
    ('Below 14 years','Total'),
    ('14 and Above – Below 18 years','Male'),
    ('14 and Above – Below 18 years','Female'),
    ('14 and Above – Below 18 years','Transgender'),
    ('14 and Above – Below 18 years','Total'),
    ('18 and Above – Below 30 years','Male'),
    ('18 and Above – Below 30 years','Female'),
    ('18 and Above – Below 30 years','Transgender'),
    ('18 and Above – Below 30 years','Total')
]
tableA.columns = pd.MultiIndex.from_tuples(tableAColumnList)
tableA.set_index(["Sl. No.","State/UT"],inplace=True)

tableBColumnList = [
    ("Sl. No.",''),
    ("State/UT",''), 
    ('30 and Above – Below 45 years','Male'),
    ('30 and Above – Below 45 years','Female'),
    ('30 and Above – Below 45 years','Transgender'),
    ('30 and Above – Below 45 years','Total'),
    ('45 and Above – Below 60 years','Male'),
    ('45 and Above – Below 60 years','Female'),
    ('45 and Above – Below 60 years','Transgender'),
    ('45 and Above – Below 60 years','Total'),
    ('60 years & Above','Male'),
    ('60 years & Above','Female'),
    ('60 years & Above','Transgender'),
    ('60 years & Above','Total')
]
tableB.columns = pd.MultiIndex.from_tuples(tableBColumnList)
tableB.set_index(["Sl. No.","State/UT"],inplace=True)

tableCColumnList = [
    ("Sl. No.",''),
    ("State/UT",''), 
    ('Total','Male'),
    ('Total','Female'),
    ('Total','Transgender'),
    ('Total','Total'),
    ('Total','Percentage Share')
]
tableC.columns = pd.MultiIndex.from_tuples(tableCColumnList)
tableC.set_index(["Sl. No.","State/UT"],inplace=True)

In [18]:
# Rename indexes to bring uniformity when joining/concatenating
tableB.index = tableA.index.copy()
tableC.index = tableA.index.copy()

In [19]:
# Create Final Table using joins on the 2 indexes we selected
final = tableA.join(tableB,on=["Sl. No.","State/UT"]).join(tableC,on=["Sl. No.","State/UT"])
final.head()

Below 14 years  \
                                                                     Male   
Sl. No. State/UT                                                            
1       Air Crash                                                       0   
2       Ship Accidents                                                  0   
3       Collapse of Structure (Total)                                 141   
        3.1 Collapse of Dwelling House/Residential Buil...            105   
        3.2 Collapse of Official/ Commercial Building                   2   

                                                                               \
                                                           Female Transgender   
Sl. No. State/UT                                                                
1       Air Crash                                               0           0   
2       Ship Accidents                                          0           0   
3       Collapse of Structure (Total)                          91           0   
        3.1 Collapse of Dwelling House/Residential Buil...     54           0   
        3.2 Collapse of Official/ Commercial Building           3           0   

                                                                  \
                                                           Total   
Sl. No. State/UT                                                   
1       Air Crash                                              0   
2       Ship Accidents                                         0   
3       Collapse of Structure (Total)                        232   
        3.1 Collapse of Dwelling House/Residential Buil...   159   
        3.2 Collapse of Official/ Commercial Building          5   

                                                           14 and Above – Below 18 years  \
                                                                                    Male   
Sl. No. State/UT                                                                           
1       Air Crash                                                                      0   
2       Ship Accidents                                                                 0   
3       Collapse of Structure (Total)                                                 83   
        3.1 Collapse of Dwelling House/Residential Buil...                            56   
        3.2 Collapse of Official/ Commercial Building                                  5   

                                                                               \
                                                           Female Transgender   
Sl. No. State/UT                                                                
1       Air Crash                                               0           0   
2       Ship Accidents                                          0           0   
3       Collapse of Structure (Total)                          38           0   
        3.1 Collapse of Dwelling House/Residential Buil...     30           0   
        3.2 Collapse of Official/ Commercial Building           1           0   

                                                                  \
                                                           Total   
Sl. No. State/UT                                                   
1       Air Crash                                              0   
2       Ship Accidents                                         0   
3       Collapse of Structure (Total)                        121   
        3.1 Collapse of Dwelling House/Residential Buil...    86   
        3.2 Collapse of Official/ Commercial Building          6   

                                                           18 and Above – Below 30 years  \
                                                                                    Male   
Sl. No. State/UT                                                                           
1       Air Crash                        

In [20]:
# Interate through entire IndexList to get the rows whose States need to be split
# e.g 
# Sl. No      State
#             7.1 Fall from Height
#         to
# Sl. No      State
# 7.1         Fall from Height

i = 0
for index, each in final["Below 14 years"].iterrows():
    print(i, index)
    i+=1

0 ('1', 'Air Crash')
1 ('2', 'Ship Accidents')
2 ('3', 'Collapse of Structure (Total)')
3 ('', '3.1 Collapse of Dwelling House/Residential Building')
4 ('', '3.2 Collapse of Official/ Commercial Building')
5 ('', '3.3 Collapse of Dam')
6 ('', '3.4 Collapse of  Bridge')
7 ('', '3.5 Others')
8 ('4', 'Drowning (Total)')
9 ('', '4.1 Boat Capsize')
10 ('', '4.2 Accidental Falls into  Waterbody')
11 ('', '4.3 Other Cases')
12 ('5', 'Electrocution')
13 ('6', 'Accidental Explosion (Total)')
14 ('', '6.1 Domestic Gas Cylinder')
15 ('', '6.2 Industrial Boiler/ Gas Cylinder Explosion')
16 ('', '6.3 Ammunition Explosion in       Armed Forces/Police/CPMF')
17 ('', '6.4 Other')
18 ('7', 'Falls (Total)')
19 ('', '7.1 Fall from Height')
20 ('', '7.2 Fall from Vehicles (Automobile like Bus, Trucks, etc.)')
21 ('', '7.3 Fall into Manhole')
22 ('', '7.4 Fall into Pit')
23 ('', '7.5 Fall into Bore well')
24 ('', '7.6 Others')
25 ('8', 'Factory/Machine Accidents')
26 ('9', 'Accidental Fire (Total)')
27 (''

Such index ranges are:
1. 3-7
2. 9-11
3. 14-17
4. 19-24
5. 27-31
6. 35-37
7. 40-41
8. 43-44
9. 48-55

In [21]:
final = final.reset_index()

In [22]:
def split_by_first_space(DF, strIndex, stpIndex):
    df = DF.iloc[strIndex:stpIndex+1].copy()
    for index, each in df.iterrows():
            space = each["State/UT"][0].find(' ')
            i = each["State/UT"][0][:space]
            val = each["State/UT"][0][space:]
            each["Sl. No."] = i
            each["State/UT"] = val
    DF[strIndex:stpIndex+1] = df.copy()
        
split_by_first_space(final,3,7)
split_by_first_space(final,9,11)
split_by_first_space(final,14,17)
split_by_first_space(final,19,24)
split_by_first_space(final,27,31)
split_by_first_space(final,35,37)
split_by_first_space(final,40,41)
split_by_first_space(final,43,44)
split_by_first_space(final,48,55)

In [23]:
final.head()

Sl. No.                                          State/UT Below 14 years  \
                                                                      Male   
0       1                                         Air Crash              0   
1       2                                    Ship Accidents              0   
2       3                     Collapse of Structure (Total)            141   
3     3.1   Collapse of Dwelling House/Residential Building            105   
4     3.2         Collapse of Official/ Commercial Building              2   

                           14 and Above – Below 18 years                     \
  Female Transgender Total                          Male Female Transgender   
0      0           0     0                             0      0           0   
1      0           0     0                             0      0           0   
2     91           0   232                            83     38           0   
3     54           0   159                            56     30           0   
4      3           0     5                             5      1           0   

         ... 45 and Above – Below 60 years 60 years & Above         \
  Total  ...                         Total             Male Female   
0     0  ...                             0                0      0   
1     0  ...                             0                0      0   
2   121  ...                           368              115     76   
3    86  ...                           224               76     56   
4     6  ...                            23                4      0   

                    Total                                            
  Transgender Total  Male Female Transgender Total Percentage Share  
0           0     0    11      1           0    12              0.0  
1           0     0     0      0           0     0              0.0  
2           0   191  1363    566           0  1929              0.5  
3           0   132   867    374           0  1241              0.3  
4           0     4    89     15           0   104              0.0  

[5 rows x 31 columns]

In [24]:
# Reset Index before saving file, for better formatting in RAW CSV
final.to_csv("Age and Gender-wise Distribution of Deaths due to Other Causes during 2019 (All India).csv",index=False)